<a href="https://colab.research.google.com/github/kg571852741/Open3D/blob/master/Open3d_Kitti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/
!git clone https://github.com/isl-org/Open3D-ML
%cd /content/Open3D-ML

/content
Cloning into 'Open3D-ML'...
remote: Enumerating objects: 6456, done.
remote: Counting objects: 100% (412/412), done.
remote: Compressing objects: 100% (300/300), done.
remote: Total 6456 (delta 247), reused 215 (delta 99), pack-reused 6044
Receiving objects: 100% (6456/6456), 37.67 MiB | 21.81 MiB/s, done.
Resolving deltas: 100% (4319/4319), done.
/content/Open3D-ML


In [ ]:
!pip install open3d -qqq
!pip install -r requirements-torch-cuda.txt -qqq

     |████████████████████████████████| 408.6 MB 30 kB/s 
     |████████████████████████████████| 596 kB 64.5 MB/s 
     |████████████████████████████████| 1.1 MB 47.4 MB/s 
     |████████████████████████████████| 8.7 MB 7.9 MB/s 
     |████████████████████████████████| 4.3 MB 45.2 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 343 kB 39.0 MB/s 
     |████████████████████████████████| 428 kB 39.8 MB/s 
     |████████████████████████████████| 1.1 MB 38.9 MB/s 
     |████████████████████████████████| 79 kB 7.8 MB/s 
     |████████████████████████████████| 130 kB 59.5 MB/s 
     |████████████████████████████████| 561 kB 46.0 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 132 kB 53.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 req

In [ ]:
%cd /content/
!python -c "import open3d.ml.torch as ml3d"

/content


In [ ]:
!mkdir /content/data/velodyne

In [ ]:
%cd /content/data/velodyne
!wget http://semantic-kitti.org/assets/data_odometry_voxels.zip

/content/data/velodyne
--2022-04-07 13:17:19--  http://semantic-kitti.org/assets/data_odometry_voxels.zip
Resolving semantic-kitti.org (semantic-kitti.org)... 131.220.233.2
Connecting to semantic-kitti.org (semantic-kitti.org)|131.220.233.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 693975024 (662M) [application/zip]
Saving to: ‘data_odometry_voxels.zip’

data_odometry_voxel 100%[===================>] 661.83M  19.1MB/s    in 74s     

2022-04-07 13:18:33 (8.94 MB/s) - ‘data_odometry_voxels.zip’ saved [693975024/693975024]



In [ ]:
%cd /content/data/velodyne
!unzip -j data_odometry_voxels.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import open3d.ml.torch as ml3d
import open3d.ml as _ml3d
import plotly.graph_objects as go
import numpy as np

In [ ]:
import os
import open3d.ml as _ml3d
import open3d.ml.torch as ml3d

cfg_file = "/content/Open3D-ML/ml3d/configs/randlanet_semantickitti.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

model = ml3d.models.RandLANet(**cfg.model)



datapath =  '/content/data/velodyne'

dataset = ml3d.datasets.KITTI(dataset_path=datapath, use_cache = True)

pipeline = ml3d.pipelines.SemanticSegmentation(model=model, dataset = dataset, **cfg.pipeline)





In [ ]:


# download the weights.
ckpt_folder = "./logs/"
os.makedirs(ckpt_folder, exist_ok=True)
ckpt_path = ckpt_folder + "randlanet_semantickitti_202201071330utc.pth"
randlanet_url = "https://storage.googleapis.com/open3d-releases/model-zoo/randlanet_semantickitti_202201071330utc.pth"
if not os.path.exists(ckpt_path):
    cmd = "wget {} -O {}".format(randlanet_url, ckpt_path)
    os.system(cmd)

# load the parameters.
pipeline.load_ckpt(ckpt_path=ckpt_path)



INFO - 2022-04-07 13:23:36,926 - semantic_segmentation - Loading checkpoint ./logs/randlanet_semantickitti_202201071330utc.pth


In [ ]:


test_split = dataset.get_split("test")
data = test_split.get_data(0)

# run inference on a single example.
# returns dict with 'predict_labels' and 'predict_scores'.
result = pipeline.run_inference(data)

In [ ]:
import numpy as np
import open3d as o3d
data_viz = {}
filepath='/content/drive/MyDrive/L001.xyz'

#point_cloud= np.loadtxt(filepath,skiprows=1)
point_cloud= np.loadtxt(filepath)
pcd = o3d.geometry.PointCloud()

pcd.points = o3d.utility.Vector3dVector(point_cloud[:,:3])
feat =  np.zeros((len(pcd.points), 3))
label = np.zeros((len(pcd.points), 1))


In [ ]:
data = {
    'point': np.asarray(pcd.points) , # your point cloud, an nx3 numpy array
    'label': label
    
      }


In [ ]:
for k, v in data.items():
  print(k, v.shape, 'split count', len(v)/100000)

point (21567172, 3) split count 215.67172
label (21567172, 1) split count 215.67172


In [ ]:

# run inference on a single example.
# returns dict with 'predict_labels' and 'predict_scores'.
result = pipeline.run_inference(data)

# evaluate performance on the test set; this will write logs to './logs'.
#pipeline.run_test()


test 0/1: 100%|██████████| 1610150/1610150 [4:04:16<00:00, 109.86it/s]

test 0/1: 100%|██████████| 1610150/1610150 [3:51:00<00:00,  8.98it/s]/usr/local/lib/python3.7/dist-packages/open3d/_ml3d/torch/modules/metrics/semseg_metric.py:54: RuntimeWarning: Mean of empty slice
  accs.append(np.nanmean(accs))
INFO - 2022-04-07 21:32:25,791 - semantic_segmentation - Accuracy : [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
/usr/local/lib/python3.7/dist-packages/open3d/_ml3d/torch/modules/metrics/semseg_metric.py:87: RuntimeWarning: Mean of empty slice
  ious.append(np.nanmean(ious))
INFO - 2022-04-07 21:32:25,794 - semantic_segmentation - IoU : [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]


In [ ]:
from numpy.lib.arraysetops import unique
pred_label = (result['predict_labels']  ).astype(np.int32)
pred_label
#len(pred_label)
#unique(pred_label)

unique(pred_label)

array([18], dtype=int32)